In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0418 22:18:15.726008170    2423 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0418 22:18:18.316833936    2423 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
bbb_itp_id = 300

In [4]:
analysis_date = dt.date(2022, 3, 30)

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

In [6]:
bbb = rt.OperatorDayAnalysis(bbb_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 

In [7]:
bbb.rt_trips >> head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,median_time,direction,mean_speed_mph,calitp_agency_name
0,300,0,2022-03-30,-8925704923997640353,883570,3498,0,25951,2022-03-29,2099-01-01,41,SMC- 17th St Station- Montana,None,07:25:25,Westbound,7.926179,Big Blue Bus
1,300,0,2022-03-30,1578699584878979521,883568,3498,0,25951,2022-03-29,2099-01-01,41,SMC- 17th St Station- Montana,None,08:08:16,Westbound,7.945861,Big Blue Bus
2,300,0,2022-03-30,-7780170951977408389,883540,3498,0,25951,2022-03-29,2099-01-01,41,SMC- 17th St Station- Montana,None,17:36:30.500000,Westbound,7.691216,Big Blue Bus


In [8]:
rt.OperatorDayAnalysis.set_filter(bbb, start_time='15:00', end_time='19:00')

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_names': None,
 'shape_ids': None,
 'direction_id': None,
 'direction': None}

In [9]:
rt.OperatorDayAnalysis.set_filter(bbb, start_time='15:00', end_time='19:00', route_names=['1', '3', 'R12'])

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_names': ['1', '3', 'R12'],
 'shape_ids': None,
 'direction_id': None,
 'direction': None}

In [10]:
m = rt.OperatorDayAnalysis.segment_speed_map(bbb)

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_names': ['1', '3', 'R12'], 'shape_ids': None, 'direction_id': None, 'direction': None}
25888_0_ empty!
25915_0_ empty!
25876_1_ empty!
speed above 80 for shape 25877, dropping
25877_0_ empty!
25887_1_ empty!
25914_1_ empty!


/home/jovyan/data-analyses/rt_delay/rt_analysis.py:606: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [11]:
m

In [12]:
rt.OperatorDayAnalysis.set_filter(bbb, start_time='15:00', end_time='19:00', shape_ids=['25404'])

IndexError: single positional indexer is out-of-bounds

In [ ]:
# bbb.set_filter(start_time='15:00', end_time='19:00', shape_ids=['25404'], direction_id='0')

In [ ]:
chart = rt.OperatorDayAnalysis.chart_variability(bbb, min_stop_seq = 20, max_stop_seq = 40)

In [ ]:
bbb.position_interpolators['860694']['rt'].direction

In [ ]:
bbb._filter(bbb.rt_trips)

### Cleaning and Formatting!
   * some sort of stop formatting? beyond optionally splitting on &?

### Files for self-serve

In [ ]:
bbb.rt_trips['calitp_agency_name'] = bbb.calitp_agency_name

In [ ]:
bbb.rt_trips.to_parquet('./temp/rt_trips.parquet')

In [ ]:
bbb.stop_segment_speed_view.to_parquet('./temp/segment_speed_view.parquet')

### Example self-serve

In [ ]:
trips = pd.read_parquet('./temp/rt_trips.parquet')

In [ ]:
segments = gpd.read_parquet('./temp/segment_speed_view.parquet')

In [ ]:
import rt_filter_map_plot

In [ ]:
importlib.reload(rt_filter_map_plot)

In [ ]:
bbb2 = rt_filter_map_plot.RtFilterMapper(trips, segments)

In [ ]:
bbb2.set_filter(start_time='15:00', end_time='19:00')

In [ ]:
# bbb2.set_filter(start_time='15:00', end_time='19:00', route_names=['1', '3', 'R12'])

In [ ]:
bbb2.show_speed_map()

In [ ]:
bbb2.stop_segment_speed_view = bbb2.stop_segment_speed_view >> filter(_.speed_mph < 70)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
bbb2.set_filter(start_time='15:00', end_time='19:00', shape_ids=['25411'])

In [ ]:
interact(bbb2.chart_variability, min_stop_seq = 25, max_stop_seq = 25)

In [ ]:
bbb2_routes = list(bbb2.rt_trips.route_short_name.unique())
all_routes = widgets.SelectMultiple(
    options=bbb2_routes,
    value=bbb2_routes[:1],
    #rows=10,
    description='Routes',
    disabled=False
)

In [ ]:
interact(bbb2.set_filter, start_time = fixed('15:00'), end_time = fixed('19:00'),
        route_names = all_routes, shape_ids = fixed(False),
        direction_id = fixed(False), direction = fixed(False))

In [ ]:
bbb2.show_speed_map()

In [ ]:
def filter_then_map(filter_mapper, start_time, end_time, route_names, shape_ids, direction_id, direction):
    filter_mapper.set_filter(start_time = start_time, end_time = end_time,
        route_names = all_routes, shape_ids = shape_ids,
        direction_id = direction_id, direction = direction)
    return filter_mapper.show_speed_map()

In [ ]:
interact(filter_then_map, filter_mapper = fixed(bbb2), start_time = fixed('15:00'), end_time = fixed('19:00'),
        route_names = all_routes, shape_ids = fixed(False),
        direction_id = fixed(False), direction = fixed(False))